## Convert baseline Outputs
Outputs generated by baselines are generally in the form of triplets. Convert them to our standard format used by evaluation scripts.

In [ ]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("..")
import pandas as pd
from pathlib import Path
from llm_food_taxonomy.data.loader import load_baseline_taxonomy
from llm_food_taxonomy.data.loader import load_taxonomy

In [ ]:
dataset = "semeval_verb"

In [ ]:
out_dir = Path("..") / "output" / dataset / "tmn"

In [ ]:
terms, taxo = load_taxonomy(f"../data/{dataset}")
id_to_name = dict(zip(terms.node_id, terms.node_name))

In [ ]:
terms.head()

In [ ]:
df = load_baseline_taxonomy(out_dir / "predicted_positions.tsv")
df.sample(20)

In [ ]:
df.shape

In [ ]:
taxo_list = [tuple(v) for v in taxo.values]
print(len(taxo_list))
unique_to_common = terms.set_index("unique_node_name")

def get_name(x, unique=False):
    if "Taxon " in x:
        x = x.split("name:")[1].split(",")[0].strip()
    if x is None or x == "leaf" or x == "pseudo leaf" or x == "pseudo root" or x == "root":
        return None
    cleaned_x = x.split("@")[0].strip().lower()
    return cleaned_x

In [ ]:
# get_name("Baking mix@@@1487")

In [ ]:
triplets = df.apply(lambda x: (get_name(x["parent"]),
                               get_name(x["query"], unique=True),
                               get_name(x["child"])),
                    axis=1)
triplets = pd.DataFrame(triplets.values.tolist(), columns=["parent", "query", "child"])
triplets = triplets[["query", "parent", "child"]]
triplets.head()

In [ ]:
triplets.shape

In [ ]:
out_dir

In [ ]:
out_dir.mkdir(parents=True, exist_ok=True)
triplets.to_csv(out_dir / "predictions.triplets", sep="\t", index=False)

with open(out_dir / "concepts.terms", "w") as f:
    for id, name in id_to_name.items():
        f.write(f"{id}\t{name}\n")

In [ ]:
print(out_dir)